In [1]:
import pandas as pd
import os
from sodapy import Socrata
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas.io.sql as psql


# # get connected to the database
# connection = pg.connect("dbname=postgres user=root password=Murphy8034?")
 
# dataframe = psql.frame_query("SELECT * FROM data_df", connection)


In [3]:
# 1. Create a DataFrame Covid data(full csv). 
# data_df = pd.read_csv('/Users/benjaminshelburn/Desktop/Classwork/Final Project/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

# 2. Create a DF from small API sample - limit 250K?
client = Socrata("data.cdc.gov", None)
data = client.get("n8mc-b4w4", limit= 500000)
data_df = pd.DataFrame.from_records(data)
data_df

# # 3. https://stackoverflow.com/questions/66345379/not-getting-full-data-from-api-to-jupyter-notebook
# r = requests.get("https://pokeapi.co/api/v2/pokemon/")
# x = r.json()

# data = []
# data.append(x['results'])

# # Loop until `next` is `None`
# while x.get('next'):
#     r = requests.get("https://pokeapi.co/api/v2/pokemon/")
#     x = r.json()
#     data.append(x['results'])

# df = pd.DataFrame(data)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn
0,2020-12,AR,05,BOONE,05009,0 - 17 years,NA,NA,NA,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,NA,NaN,NaN
1,2021-10,MO,29,PHELPS,29161,0 - 17 years,NA,NA,NA,0,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,Unknown,NaN,NaN
2,2020-10,MO,29,CASS,29037,0 - 17 years,Female,NA,NA,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Unknown,Missing,Unknown,0,NaN
3,2021-08,TN,47,CLAIBORNE,47025,0 - 17 years,Female,Missing,Missing,0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN,NaN
4,2020-11,GA,13,EFFINGHAM,13103,0 - 17 years,Female,Unknown,Missing,NaN,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2021-09,GA,13,COFFEE,13069,0 - 17 years,Female,White,Non-Hispanic/Latino,NaN,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,0,NaN
499996,2021-09,CA,06,DEL NORTE,06015,0 - 17 years,Female,White,Non-Hispanic/Latino,NaN,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,Missing,0,NaN
499997,2021-01,GA,13,MURRAY,13213,0 - 17 years,Female,White,Non-Hispanic/Latino,NaN,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,0,NaN
499998,2021-08,OH,39,PORTAGE,39133,0 - 17 years,Female,White,Non-Hispanic/Latino,0,Clinical evaluation,Yes,Probable Case,Symptomatic,No,No,No,0,No


In [4]:
# Columns to drop
data_df = data_df.drop(['state_fips_code'], axis=1)
data_df = data_df.drop(['county_fips_code'], axis=1)
data_df = data_df.drop(['case_positive_specimen'], axis=1)
data_df = data_df.drop(['current_status'], axis=1)
data_df = data_df.drop(['case_onset_interval'], axis=1)
data_df = data_df.drop(['ethnicity'], axis=1)
data_df = data_df.drop(['process'], axis=1)
data_df = data_df.drop(['res_county'], axis=1)


# Print df
data_df

,case_month,res_state,age_group,sex,race,exposure_yn,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-12,AR,0 - 17 years,NA,NA,Missing,Symptomatic,Missing,Missing,NA,NaN
1,2021-10,MO,0 - 17 years,NA,NA,Missing,Missing,Unknown,Missing,Unknown,NaN
2,2020-10,MO,0 - 17 years,Female,NA,Missing,Symptomatic,Unknown,Missing,Unknown,NaN
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...
499995,2021-09,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN
499996,2021-09,CA,0 - 17 years,Female,White,Missing,Symptomatic,No,Missing,Missing,NaN
499997,2021-01,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN
499998,2021-08,OH,0 - 17 years,Female,White,Yes,Symptomatic,No,No,No,No


In [5]:
# Change and format column names
data_df.rename(columns={'case_month':'Case Month', 'res_state':'Resident State', 'res_county': 'Resident County',
                       'age_group':'Age Group', 'sex':'Sex', 'race':'Race', 'ethnicity': 'Ethnicity',
                       'process':'Process', 'exposure_yn': 'Exposure', 'symptom_status': 'Symptom Status',
                       'hosp_yn':'Hospitalization', 'icu_yn': 'ICU', 'death_yn':'Death',
                        'underlying_conditions_yn': 'Underlying Health Conditions'},inplace=True )

In [6]:
# Copy df and insert blank Final Outcome column
data_df = data_df.copy()
data_df['Final Outcome']=""
data_df

,Case Month,Resident State,Age Group,Sex,Race,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,0 - 17 years,NA,NA,Missing,Symptomatic,Missing,Missing,NA,NaN,
1,2021-10,MO,0 - 17 years,NA,NA,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,0 - 17 years,Female,NA,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN,
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2021-09,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
499996,2021-09,CA,0 - 17 years,Female,White,Missing,Symptomatic,No,Missing,Missing,NaN,
499997,2021-01,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
499998,2021-08,OH,0 - 17 years,Female,White,Yes,Symptomatic,No,No,No,No,


In [7]:
# remove rows using the drop() function
data_df.drop(data_df.index[data_df['Resident State'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Resident County'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Age Group'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Age Group'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Symptom Status'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Symptom Status'] == 'Missing'], inplace=True)

#data_df.drop(data_df.index[data_df['Ethnicity'] == 'Missing'], inplace=True)

# display the dataframe
data_df

,Case Month,Resident State,Age Group,Sex,Race,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,0 - 17 years,NA,NA,Missing,Symptomatic,Missing,Missing,NA,NaN,
1,2021-10,MO,0 - 17 years,NA,NA,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,0 - 17 years,Female,NA,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN,
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2021-09,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
499996,2021-09,CA,0 - 17 years,Female,White,Missing,Symptomatic,No,Missing,Missing,NaN,
499997,2021-01,GA,0 - 17 years,Female,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
499998,2021-08,OH,0 - 17 years,Female,White,Yes,Symptomatic,No,No,No,No,


In [8]:
data_df.to_csv(r'/Users/benjaminshelburn/Desktop/Classwork/Final Project/CleanData.csv', index = False)


In [9]:
data_df.count()

Case Month                      500000
Resident State                  500000
Age Group                       500000
Sex                             500000
Race                            500000
Exposure                        500000
Symptom Status                  500000
Hospitalization                 500000
ICU                             500000
Death                           500000
Underlying Health Conditions     39472
Final Outcome                   500000
dtype: int64

In [15]:
data_df = data_df[["Case Month","Resident State"]]
data_df

,Case Month,Resident State
0,2020-12,AR
1,2021-10,MO
2,2020-10,MO
3,2021-08,TN
4,2020-11,GA
...,...,...
499995,2021-09,GA
499996,2021-09,CA
499997,2021-01,GA
499998,2021-08,OH
